In [75]:
import vertexai
from vertexai.preview.language_models import ChatModel, InputOutputTextPair
from IPython.display import display, Markdown

In [76]:
vertexai.init(project="synap-labs-390404", location="us-central1")

chat_model = ChatModel.from_pretrained("chat-bison@001")

parameters = {
    "temperature": 0,
    "max_output_tokens": 1024,
    "top_p": 0.3,
    "top_k": 40
}

In [77]:
chat = chat_model.start_chat(
    context="""You are a smart product manager who answers in a concise way. You have to help the user create a Product Requirement Document based on the questions the users asks you. The user will ask you specific questions about each topic they want to be included in the PRD. 
    
    You have to only answer the questions asked by the user, and not provide any additional information. This is a very important skill for a product manager, as they have to be concise and to the point. 
    
    Format your responses in Markdown mode with each topic being the ##heading, and your answer being the content. Highlight importand points in **bold**""",
)

In [78]:
response = chat.send_message("""I want to add the following new feature:
Instagram Threads

Feature description: Gives users the ability to create tweets-like posts on instagram.

1. Purpose & Scope: 
- An overview of the product
- Define the target audience in detail
- Define the stakeholders
- Explain the gap it is trying to fill""", **parameters)

prd = response.text + "\n\n"

In [79]:
response = chat.send_message("""2. Product Objectives:
- How do the objectives of this product align with the company\'s objectives?
- The business style of the PRD should be similar to the business style of the company""", **parameters)

prd += response.text + "\n\n"

In [80]:
response = chat.send_message("""Market Research:
- How is the target audience in this segment?
- What are the industry trends and customer expectations?""", **parameters)

# prd += response.text + "\n\n"

In [81]:
response = chat.send_message("""List some competitors of this application in bullet points.""")

prd += response.text + "\n\n"

In [82]:
response = chat.send_message("""Use all the above competitors to create a competitive analysis of these applications in a tabular form using the following points - user base, user region, different features supported, and pricing tiers. Don't limit yourself to these categories and think of other categories yourself. Return the output in a well-structured Markdown table""")

prd += response.text + "\n\n"

In [83]:
response = chat.send_message("""Feature Requirements:
- What are some of the important features that should be implemented?
- Follow the MoSCoW format (Must have, Should have, Could have, Won’t have).
- If you have any questions for the user, you can ask them here.""", **parameters)

prd += response.text + "\n\n"

In [84]:
response = chat.send_message("""Acceptance Criteria:
- Define the technical and non-technical quality of completeness required to be able to get to the MVP stage of this product""", **parameters)

prd += response.text + "\n\n"

In [85]:
response = chat.send_message("""Success Metrics:
- How do we define success in this product?
- What are the KPIs to look out for?""", **parameters)

prd += response.text + "\n\n"

In [86]:
response = chat.send_message("""Technical Feasibilities:
- What devices should this app be available for? Web/iOS/Android/more
- What is the tech stack to be used for the frontend?
- What is the tech stack to be used for making the backend scalable?""", **parameters)

prd += response.text + "\n\n"

In [87]:
with open("../generated_prds/Instagram Threads Chat VertexAI PRD.md", "w") as f:
    f.write(prd)

In [88]:
display(Markdown(prd))

## Purpose & Scope

**Overview**

Instagram Threads is a new feature that will allow users to create tweets-like posts on Instagram. This will allow users to share their thoughts and ideas in a more concise and easy-to-read format.

**Target Audience**

The target audience for Instagram Threads is millennials and Gen Z users who are looking for a more engaging way to share their thoughts and ideas.

**Stakeholders**

The stakeholders for Instagram Threads include Instagram's users, as well as the company itself. Instagram's users will benefit from being able to share their thoughts and ideas in a more concise and easy-to-read format. Instagram will benefit from the increased engagement and user activity that Instagram Threads will generate.

**Gap it is trying to fill**

Instagram Threads is trying to fill the gap in Instagram's current offering by providing a way for users to share their thoughts and ideas in a more concise and easy-to-read format. Instagram currently does not have a feature that allows users to do this, and as a result, users are often forced to use other platforms, such as Twitter, to share their thoughts and ideas. Instagram Threads will allow users to stay on Instagram and share their thoughts and ideas with their followers.

## Product Objectives

Instagram Threads is aligned with Instagram's overall objective of providing a platform for users to share their thoughts and ideas. Instagram Threads will allow users to do this in a more concise and easy-to-read format, which will increase engagement and user activity.

Instagram Threads is also aligned with Instagram's objective of staying ahead of the competition. Twitter is a popular platform for users to share their thoughts and ideas, and Instagram Threads will allow Instagram to compete with Twitter in this area.

The business style of the PRD should be similar to the business style of the company. This means that the PRD should be concise, clear, and easy to understand. The PRD should also be focused on the business objectives of the product.

## Competitors

* Twitter
* Facebook
* Snapchat
* TikTok

These platforms all allow users to share their thoughts and ideas in a concise and easy-to-read format. Instagram Threads will compete with these platforms by offering a unique and engaging experience that is tailored to the needs of millennials and Gen Z users.

## Competitive Analysis

| Competitors | User Base | User Region | Different Features Supported | Pricing Tiers |
|---|---|---|---|---|
| Instagram Threads | Millennials and Gen Z users | Worldwide | Tweets-like posts, photos, videos, stories, direct messages, live videos, reels | Free |
| Twitter | All ages | Worldwide | Tweets, photos, videos, direct messages, live videos, polls | Free |
| Facebook | All ages | Worldwide | Posts, photos, videos, direct messages, live videos, stories, events | Free |
| Snapchat | Millennials and Gen Z users | Worldwide

## Feature Requirements

**Must Have**

* Users must be able to create tweets-like posts on Instagram.
* Tweets-like posts must be limited to 280 characters.
* Tweets-like posts must be visible to the user's followers.
* Users must be able to like, comment, and share tweets-like posts.

**Should Have**

* Users should be able to add photos, videos, and links to their tweets-like posts.
* Users should be able to tag other users in their tweets-like posts.
* Users should be able to schedule tweets-like posts to be published at a later date.

**Could Have**

* Users could have the ability to create polls in their tweets-like posts.
* Users could have the ability to create quizzes in their tweets-like posts.
* Users could have the ability to create events in their tweets-like posts.

**Won't Have**

* Users will not be able to delete tweets-like posts after they have been published.
* Users will not be able to edit tweets-like posts after they have been published.

**Questions for the user:**

* What are the key features that you would like to see in Instagram Threads?
* What are the features that you would consider to be essential?
* What are the features that you would consider to be nice-to-have?
* What are the features that you would consider to be unnecessary?

## Acceptance Criteria

The following are the acceptance criteria for Instagram Threads:

* **Technical:**
    * The feature must be implemented in a way that is scalable and reliable.
    * The feature must be secure and protect user data.
    * The feature must be accessible to users with disabilities.
* **Non-technical:**
    * The feature must be easy to use and understand.
    * The feature must be engaging and interesting to users.
    * The feature must be aligned with Instagram's overall brand and values.

These acceptance criteria will be used to evaluate the success of Instagram Threads and to determine when the feature is ready to be released to the public.

## Success Metrics

The following are the success metrics for Instagram Threads:

* **Number of active users:** The number of users who use Instagram Threads on a regular basis.
* **Number of tweets-like posts:** The number of tweets-like posts that are created on Instagram Threads.
* **Engagement rate:** The percentage of users who like, comment, or share tweets-like posts.
* **Retention rate:** The percentage of users who continue to use Instagram Threads after the first month.

These metrics will be used to evaluate the success of Instagram Threads and to determine whether the feature is meeting the company's objectives.

## Technical Feasibilities

* **Devices**

Instagram Threads will be available for iOS and Android devices.

* **Frontend**

The frontend of Instagram Threads will be built using React Native.

* **Backend**

The backend of Instagram Threads will be built using a microservices architecture. The microservices will be hosted on AWS Lambda.

* **Scalability**

Instagram Threads will be scalable to handle a large number of users. The microservices will be load balanced and auto scaled to meet demand.

* **Security**

Instagram Threads will be secure and protect user data. The data will be encrypted at rest and in transit.

* **Accessibility**

Instagram Threads will be accessible to users with disabilities. The feature will be compliant with the Web Content Accessibility Guidelines (WCAG).

* **Localization**

Instagram Threads will be localized for a variety of languages. The feature will be translated into the following languages: English, Spanish, French, German, Italian, Portuguese, Russian, Chinese, Japanese, and Korean.

* **Testing**

Instagram Threads will be thoroughly tested before it is released to the public. The feature will be tested for functionality, performance, security, and accessibility.

* **Deployment**

Instagram Threads will be deployed to production using a continuous integration and continuous delivery (CI/CD) pipeline. The feature will be deployed to a staging environment before it is deployed to production.

* **Monitoring**

Instagram Threads will be monitored to ensure that it is functioning properly. The feature will be monitored for performance, errors, and security vulnerabilities.

* **Support**

Instagram Threads will provide support to users who have questions or problems with the feature. The support team will be available to answer questions, troubleshoot problems, and provide assistance.

